In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv

In [2]:
pd.set_option('max_colwidth', 800)

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text 
soup = BeautifulSoup(source, 'lxml')

In [4]:
csv_file = open('toronto_postal_codes.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Postcode', 'Borough', 'Neighbourhood'])

32

In [5]:
table = soup.find('table', class_ = 'wikitable sortable') # Gets the table from the webpage
rows = table.find_all('tr') # Gets the table rows

postcodes = [] # Initializes the raw postcodes list
boroughs = [] # Initializes the raw boroughs list
neighbourhoods = [] # Initializes the raw neighbourhoods list

for row in rows:    
    columns = row.find_all('td')
    try :
        if columns[1].text != 'Not assigned\n':  # To skip if the borough name is 'Not Assigned'
            
            postcode = columns[0].text
            postcodes.append(postcode)
            
            borough = columns[1].text
            boroughs.append(borough)
            
            neighbourhood = columns[2].text.split('\n')[0] # Removing the newline character at the end     
            
            if neighbourhood == 'Not assigned': # Assigning the same name to neighbourhood if it is 'Not Assigned'
                neighbourhood = borough            
                
            neighbourhoods.append(neighbourhood)
             
    except Exception as e : # To skip the first row which contains column names
        pass 
    
postcode_explored = [] # Initializing the list of explored postcodes
for index_i, postcode_i in enumerate(postcodes) :   
    if postcode_i not in postcode_explored :
        nbds = neighbourhoods[index_i]
        for index_f, postcode_f in enumerate(postcodes) :
            if postcode_i == postcode_f and index_i != index_f:
                nbds = nbds + ', ' + neighbourhoods[index_f] # Concatenating the neighbourhood names
        csv_writer.writerow([postcode_i, boroughs[index_i], nbds]) # Writing the rows in the csv file
        postcode_explored.append(postcode_i)

In [6]:
csv_file.close()

In [7]:
df = pd.read_csv('toronto_postal_codes.csv')

In [8]:
df.shape

(103, 3)

In [9]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A\n,North York\n,Parkwoods
1,M4A\n,North York\n,Victoria Village
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government"


In [10]:
postal_codes = df['Postcode'].values

In [11]:
import json

latitudes = [] # Initializing the latitude array
longitudes = [] # Initializing the longitude array

for postal_code in postal_codes : 
    place_name = postal_code + " Toronto" # Formats the place name
    url = 'https://api.opencagedata.com/geocode/v1/json?q={}&key=77d6d969c7914004ae810639e74a41cd'.format(place_name) # Gets the proper url to make the API call
    obj = json.loads(requests.get(url).text) # Loads the JSON file in the form of a python dictionary
    
    results = obj['results'] # Extracts the results information out of the JSON file
    lat = results[0]['geometry']['lat'] # Extracts the latitude value
    lng = results[0]['geometry']['lng'] # Extracts the longitude value
    
    latitudes.append(lat) # Appending to the list of latitudes
    longitudes.append(lng) # Appending to the list of longitudes

In [12]:
df['Latitude'] = latitudes
df['Longitude'] = longitudes

In [13]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A\n,North York\n,Parkwoods,-33.013441,151.594204
1,M4A\n,North York\n,Victoria Village,43.727600,-79.314800
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront",43.655500,-79.362600
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights",43.722300,-79.450400
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government",49.865306,14.472511


In [14]:
# !conda install -c conda-forge folium=0.5.0 --yes
import folium # Map plotting library
import numpy as np
from pandas.io.json import json_normalize # Tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering stage
from sklearn.cluster import KMeans

In [15]:
CLIENT_ID = 'WQXS5RMB24A5EVJC05UDBIA3XXJA2JBTUAR2KEZJUJD2D5K5' 
CLIENT_SECRET = 'CJOAPMRW3LG4UCHSZIJWZU2Y42UOMQ2DF14I05ONO2UYU5CN'
VERSION = '20180604'

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: WQXS5RMB24A5EVJC05UDBIA3XXJA2JBTUAR2KEZJUJD2D5K5
CLIENT_SECRET:CJOAPMRW3LG4UCHSZIJWZU2Y42UOMQ2DF14I05ONO2UYU5CN


***Creating a folium map of Toronto***

In [16]:
# Toronto latitude and longitude using Google search
tor_lat = 43.6532
tor_lng = -79.3832

# Creates map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[tor_lat, tor_lng], zoom_start=11)

# Add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

***Defining own function to get the name of the category***

In [17]:
# Gets the name of the category

def get_category_type(row):
    try:
        categories_list = row['Category']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

***Using the FourSquare API on all neighbourhoods***

In [18]:
explore_df_list = []

for i, nbd_name in enumerate(df['Neighbourhood']):  
    
    try :         
        ### Getting the data of neighbourhood
        nbd_name = df.loc[i, 'Neighbourhood']
        nbd_lat = df.loc[i, 'Latitude']
        nbd_lng = df.loc[i, 'Longitude']

        radius = 500 # Setting the radius as 500 metres
        LIMIT = 100 # Getting the top 100 venues

        url = 'https://api.foursquare.com/v2/venues/explore?client_id={} \
        &client_secret={}&ll={},{}&v={}&radius={}&limit={}'\
        .format(CLIENT_ID, CLIENT_SECRET, nbd_lat, nbd_lng, VERSION, radius, LIMIT)

        results = json.loads(requests.get(url).text)
        results = results['response']['groups'][0]['items']

        nearby = json_normalize(results) # Flattens JSON

        # Filtering the columns
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        nearby = nearby.loc[:, filtered_columns]

        # Renaming the columns
        columns = ['Name', 'Category', 'Latitude', 'Longitude']
        nearby.columns = columns

        # Gets the categories
        nearby['Category'] = nearby.apply(get_category_type, axis=1)

        # Gets the data required
        for i, name in enumerate(nearby['Name']):
            explore_df_list.append([nbd_name, nbd_lat, nbd_lng] + nearby.loc[i, :].values.tolist())
    
    except Exception as e:
        pass

***Creating a dataframe for performing clustering operations***

In [19]:
explore_df = pd.DataFrame([item for item in explore_df_list])
explore_df.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue Name', 'Venue Category', 'Venue Latitude', 'Venue Longitude']
explore_df.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue Name,Venue Category,Venue Latitude,Venue Longitude
0,Parkwoods,-33.013441,151.594204,Subway,Sandwich Place,-33.013180,151.592120
1,Parkwoods,-33.013441,151.594204,Woolworths,Supermarket,-33.014511,151.595918
2,Parkwoods,-33.013441,151.594204,Doubletake Cafe,Café,-33.014280,151.597680
3,Parkwoods,-33.013441,151.594204,Coles,Supermarket,-33.012820,151.595456
4,Parkwoods,-33.013441,151.594204,Chantilly MediSpa,Health & Beauty Service,-33.013980,151.595480


***Total number of unique categories***

In [20]:
len(explore_df['Venue Category'].unique())

256

***Performing one-hot encoding to analyze neighbourhoods***

In [21]:
toronto_onehot = pd.get_dummies(explore_df[['Venue Category']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = explore_df['Neighbourhood'] 

# Move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Xinjiang Restaurant,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


***Aggregating venues by neighbourhoods***

In [22]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Xinjiang Restaurant,Yoga Studio
0,Agincourt,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.111111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.041667,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


***Creating a function to return the most common venues***

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

***Creating a new dataframe to get the top 10 venues***

In [24]:

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Badminton Court,Breakfast Spot,Latin American Restaurant,Yoga Studio,Filipino Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish Market
1,"Alderwood, Long Branch",Pizza Place,Dance Studio,Coffee Shop,Gym,Pub,Athletics & Sports,Sandwich Place,Convenience Store,Pharmacy,Fish & Chips Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant,Fried Chicken Joint,Deli / Bodega,Coffee Shop,Yoga Studio,Filipino Restaurant,Farmers Market,Fast Food Restaurant
3,Bayview Village,Gas Station,Flower Shop,Trail,Park,Field,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Restaurant,Coffee Shop,Pizza Place,Sandwich Place,Grocery Store,Liquor Store,Pub,Café,Thai Restaurant


***Using K-Means clustering***

In [25]:
# Set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(toronto_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# Add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

***Creating a grand dataframe containing the necessary data***

In [26]:
toronto_merged = df
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.dropna(inplace = True)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A\n,North York\n,Parkwoods,-33.013441,151.594204,2,Café,Supermarket,Pharmacy,Health & Beauty Service,Sandwich Place,Coffee Shop,Gas Station,Fast Food Restaurant,Harbor / Marina,Waterfront
1,M4A\n,North York\n,Victoria Village,43.727600,-79.314800,2,Pizza Place,Portuguese Restaurant,French Restaurant,Park,Intersection,Coffee Shop,Hockey Arena,Curling Ice,Dance Studio,Flea Market
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront",43.655500,-79.362600,2,Coffee Shop,Restaurant,Breakfast Spot,Beer Store,Electronics Store,Event Space,Thai Restaurant,Theater,Bakery,Pub
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights",43.722300,-79.450400,2,Clothing Store,Coffee Shop,Women's Store,Restaurant,Sushi Restaurant,Shoe Store,Toy / Game Store,Jewelry Store,Food Court,Bakery
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government",49.865306,14.472511,2,Campground,Castle,Yoga Studio,Fountain,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop


***Visualizing the clusters***

In [27]:
# Create map
map_clusters = folium.Map(location=[tor_lat, tor_lng], zoom_start=11)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' (Cluster ' + str(cluster) + ')', parse_html=True)
    map_clusters.add_child(
        folium.features.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7))
       
map_clusters

***Cluster 0***

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,York\n,Caledonia-Fairbanks,0,Park,Women's Store,Sporting Goods Shop,Mexican Restaurant,Bakery,Beer Store,Gym,Yoga Studio,Field,Event Space
27,North York\n,Hillcrest Village,0,Residential Building (Apartment / Condo),Park,Yoga Studio,Fast Food Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Field,Dumpling Restaurant
32,Scarborough\n,Scarborough Village,0,Grocery Store,Park,Yoga Studio,Fast Food Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Field,Dumpling Restaurant
35,East York\n,"East Toronto, Broadview North (Old East York)",0,Convenience Store,Massage Studio,Park,Yoga Studio,Filipino Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Fish Market
39,North York\n,Bayview Village,0,Gas Station,Flower Shop,Trail,Park,Field,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Yoga Studio
61,Central Toronto\n,Lawrence Park,0,Photography Studio,Park,Yoga Studio,Filipino Restaurant,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop
64,York\n,Weston,0,Convenience Store,Park,Yoga Studio,Filipino Restaurant,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop
66,North York\n,York Mills West,0,Convenience Store,Park,Yoga Studio,Filipino Restaurant,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop
72,North York\n,"Willowdale, Willowdale West",0,Coffee Shop,Park,Yoga Studio,Field,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Eastern European Restaurant
73,Central Toronto\n,"North Toronto West, Lawrence Park",0,Playground,Gym Pool,Garden,Park,Field,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Filipino Restaurant


***Cluster 1***

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Central Toronto\n,"Forest Hill North & West, Forest Hill Road Park",1,Trail,Yoga Studio,Eastern European Restaurant,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant


***Cluster 2***

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York\n,Parkwoods,2,Café,Supermarket,Pharmacy,Health & Beauty Service,Sandwich Place,Coffee Shop,Gas Station,Fast Food Restaurant,Harbor / Marina,Waterfront
1,North York\n,Victoria Village,2,Pizza Place,Portuguese Restaurant,French Restaurant,Park,Intersection,Coffee Shop,Hockey Arena,Curling Ice,Dance Studio,Flea Market
2,Downtown Toronto\n,"Regent Park, Harbourfront",2,Coffee Shop,Restaurant,Breakfast Spot,Beer Store,Electronics Store,Event Space,Thai Restaurant,Theater,Bakery,Pub
3,North York\n,"Lawrence Manor, Lawrence Heights",2,Clothing Store,Coffee Shop,Women's Store,Restaurant,Sushi Restaurant,Shoe Store,Toy / Game Store,Jewelry Store,Food Court,Bakery
4,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government",2,Campground,Castle,Yoga Studio,Fountain,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
5,Etobicoke\n,"Islington Avenue, Humber Valley Village",2,Pharmacy,Park,Skating Rink,Grocery Store,Bank,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Eastern European Restaurant
6,Scarborough\n,"Malvern, Rouge",2,Café,Supermarket,Pharmacy,Health & Beauty Service,Sandwich Place,Coffee Shop,Gas Station,Fast Food Restaurant,Harbor / Marina,Waterfront
7,North York\n,Don Mills,2,Pizza Place,Restaurant,Bank,Coffee Shop,Shoe Store,Sushi Restaurant,Café,Italian Restaurant,Cantonese Restaurant,Furniture / Home Store
8,East York\n,"Parkview Hill, Woodbine Gardens",2,Pizza Place,Intersection,Breakfast Spot,Gastropub,Gym / Fitness Center,Bank,Pet Store,Pharmacy,Dance Studio,Electronics Store
9,Downtown Toronto\n,"Garden District, Ryerson",2,Coffee Shop,Clothing Store,Café,Hotel,Cosmetics Shop,Italian Restaurant,Japanese Restaurant,Bookstore,Fast Food Restaurant,Plaza


***Cluster 3***

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Downtown Toronto\n,Berczy Park,3,Home Service,Lawyer,Filipino Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Yoga Studio
30,Downtown Toronto\n,"Richmond, Adelaide, King",3,Home Service,Lawyer,Filipino Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Yoga Studio
33,North York\n,"Fairview, Henry Farm, Oriole",3,Home Service,Lawyer,Filipino Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Yoga Studio
36,Downtown Toronto\n,"Harbourfront East, Union Station, Toronto Islands",3,Home Service,Lawyer,Filipino Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Yoga Studio
43,West Toronto\n,"Brockton, Parkdale Village, Exhibition Place",3,Home Service,Lawyer,Filipino Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Yoga Studio
59,North York\n,"Willowdale, Willowdale East",3,Home Service,Lawyer,Filipino Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Yoga Studio
62,Central Toronto\n,Roselawn,3,Clothing Store,Home Service,Eastern European Restaurant,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant
63,York\n,"Runnymede, The Junction North",3,Home Service,Lawyer,Filipino Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Yoga Studio
69,West Toronto\n,"High Park, The Junction South",3,Home Service,Lawyer,Filipino Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Yoga Studio
81,West Toronto\n,"Runnymede, Swansea",3,Home Service,Lawyer,Filipino Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Yoga Studio


***Cluster 4***

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Scarborough\n,Cedarbrae,4,Gaming Cafe,Lounge,Yoga Studio,Electronics Store,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
